# Bank Churn Prediction Proyect

## Introduction

En este proyecto vamos a analizar algunos datos del banco "Beta Bank" ya que los clientes se estan saliendo del banco poco a poco, asiq ue nuestro trabajo sera predecir si alguno de 
los clientes actuales dejara el banco pronto para asi poder tomar medidas y evitar que esto suceda. Crearemos un modelo con el maximo F1 posible.



## Objectives

Desarrollar un modelo predictivo eficiente que identifique con alta precisión a los clientes con mayor riesgo de abandonar los servicios de Beta Bank. Este modelo ayudará a la entidad bancaria a implementar estrategias proactivas para la retención de clientes, optimizando recursos y mejorando la satisfacción del cliente.

## Importacion de datos

In [57]:
# Manipulación y análisis de datos
import pandas as pd
import numpy as np

# Visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesamiento de datos
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.utils import shuffle


# Modelos de machine learning
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier


## Carga de data sets

In [58]:
df = pd.read_csv('datasets/Churn.csv')

## Analisis y preparacion de datos

### Exploracion Inicial

In [59]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Características
- RowNumber: índice de cadena de datos
- CustomerId: identificador de cliente único
- Surname: apellido
- CreditScore: valor de crédito
- Geography: país de residencia
- Gender: sexo
- Age: edad
- Tenure: período durante el cual ha madurado el depósito a plazo fijo de un cliente (años)
- Balance: saldo de la cuenta
- NumOfProducts: número de productos bancarios utilizados por el cliente
- HasCrCard: el cliente tiene una tarjeta de crédito (1 - sí; 0 - no)
- IsActiveMember: actividad del cliente (1 - sí; 0 - no)
- EstimatedSalary: salario estimado

Objetivo
- Exited: El cliente se ha ido (1 - sí; 0 - no)

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Podemos ver tenemos algunos datos nulos en Tenure, exactamente 909 los cuales representan un 9% de los datos, no es un porcentaje muy alto, pero si lo suficiente como para tener que tomar una decision sobre ellos ya que no podemos dejarlos asi, ya que podrian afectar a nuestro modelo. 

Tenemos varias opciones las cuales son las siguientes:

- Eliminarlos
- Rellenarlos con la media, mediana o moda
- Rellenarlos con un valor aleatorio

Decidimos eliminarlos ya que no es un porcentaje muy alto y no afectara mucho a nuestro modelo, y asi no tendremos que preocuparnos de ellos.

### Eliminacion de datos nulos

In [61]:
df = df.dropna(subset=['Tenure'])

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9091 entries, 0 to 9998
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        9091 non-null   int64  
 1   CustomerId       9091 non-null   int64  
 2   Surname          9091 non-null   object 
 3   CreditScore      9091 non-null   int64  
 4   Geography        9091 non-null   object 
 5   Gender           9091 non-null   object 
 6   Age              9091 non-null   int64  
 7   Tenure           9091 non-null   float64
 8   Balance          9091 non-null   float64
 9   NumOfProducts    9091 non-null   int64  
 10  HasCrCard        9091 non-null   int64  
 11  IsActiveMember   9091 non-null   int64  
 12  EstimatedSalary  9091 non-null   float64
 13  Exited           9091 non-null   int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.0+ MB


### Comprobacion de duplicados

In [63]:
print("Cantidad de duplicados =", df.duplicated().sum())


Cantidad de duplicados = 0


Eliminamos los datos nulos y comprobamos que ya no tenemos ninguno.

A demas verificamos si no tenemos valores duplicados, ya que podrian afectar a nuestro modelo.

### Eliminacion de columnas innecesarias

Ya que tenemos columnas que no van a ayudar a nuestro modelo, las vamos a eliminar, estas son RowNumber, Surname y CustomerId.

In [64]:
df = df.drop(columns=['Surname'])
df = df.drop(columns=['RowNumber'])
df = df.drop(columns=['CustomerId'])



## Categorizacion de datos

Ya que tenemos datos categoricos, tenemos que convertirlos a numericos para que nuestro modelo pueda trabajar con ellos. Especificamente Gender y Geography.

In [65]:
df = pd.get_dummies(df, columns=['Geography', 'Gender'])

df.head(5)


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,True,False,False,True,False
1,608,41,1.0,83807.86,1,0,1,112542.58,0,False,False,True,True,False
2,502,42,8.0,159660.80,3,1,0,113931.57,1,True,False,False,True,False
3,699,39,1.0,0.00,2,0,0,93826.63,0,True,False,False,True,False
4,850,43,2.0,125510.82,1,1,1,79084.10,0,False,False,True,True,False


## Division del conjunto de Datos

En este paso vamos a dividir nuestro conjunto de datos en 3 partes, train, validacion y test. Para poder entrenar nuestro modelo, validar que funciona correctamente y por ultimo testearlo.

In [66]:
df_train, df_temp = train_test_split(df, test_size=0.2, random_state=12345)
df_val, df_test = train_test_split(df_temp, test_size=0.25, random_state=12345)


## Escalado de Caracteristicas

Dado que algunas de nuestras columnas tienen valores muy altos y otras muy bajos, tenemos que escalarlas para que nuestro modelo pueda trabajar con ellas correctamente. Entre ellas tenemos CreditScore, Age, Tenure, Balance y EstimatedSalary.

In [67]:
scaler = StandardScaler()
num_columns = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler.fit(df_train[num_columns])


df_train[num_columns] = scaler.transform(df_train[num_columns])
df_val[num_columns] = scaler.transform(df_val[num_columns])
df_test[num_columns] = scaler.transform(df_test[num_columns])

## Verificacion del equilibrio de clases

In [68]:
class_distribution = df_train['Exited'].value_counts(normalize=True)
print(class_distribution)


Exited
0    0.795792
1    0.204208
Name: proportion, dtype: float64


Podemos ver que tenemos un desequilibrio de clases, ya que tenemos un 80% de clientes que no se han ido y un 20% que si se han ido. Esto puede afectar a nuestro modelo, ya que podria aprender a predecir que todos los clientes no se van a ir y tendriamos un modelo que no nos serviria para nada.

Es por eso que tenemos que balancear las clases, para que nuestro modelo aprenda a predecir correctamente.

En este caso vamos a utilizar la tecnica de upsampling, la cual consiste en duplicar los datos de la clase minoritaria.

In [77]:
X_train = df_train.drop('Exited', axis=1)
y_train = df_train['Exited']

minority_class = df_train[df_train['Exited'] == 1]

# Duplicar por 3 la clase minoritaria para o
oversampled_minority_class = pd.concat([minority_class] * 3, ignore_index=True)

# Aleatorizar el orden de las filas (shuffle)
oversampled_minority_class = shuffle(oversampled_minority_class, random_state=12345)

# Combinar las filas duplicadas con las originales
X_train_balanced = pd.concat([X_train, oversampled_minority_class.drop('Exited', axis=1)], ignore_index=True)
y_train_balanced = pd.concat([y_train, oversampled_minority_class['Exited']], ignore_index=True)

Ahora para corroborar vamos a ver el balance de clases de nuevo.

In [78]:
class_distribution_balanced = y_train_balanced.value_counts(normalize=True)
print(class_distribution_balanced)

Exited
1    0.506523
0    0.493477
Name: proportion, dtype: float64


Podemos ver que ahora tenemos un 50% de clientes que se han ido y un 50% que no se han ido, lo cual es perfecto para nuestro modelo.